<a href="https://colab.research.google.com/github/sanaaria/Master-thesis/blob/main/MPNN_CNN_%26_DeepDTI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
################################
#This script provide a demo of MPNN_CNN & DeepDTI, the runtime on one fold mainly takes 3~5 hours (V100).


In [3]:
pip install DeepPurpose

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.5/158.5 kB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 350.7/350.7 kB 27.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 53.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.1/226.1 kB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 596.4/596.4 kB 48.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 732.8/732.8 kB 48.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.1/266.1 kB 32.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.5/174.5 kB 23.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.0/91.0 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 73.7 MB/s eta 0:00:00
  Created wheel for DeepPurpose: filename=DeepPurpose-0.1.

In [4]:
pip install rdkit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.5/30.5 MB 45.5 MB/s eta 0:00:00


In [5]:
pip install git+https://github.com/bp-kelley/descriptastorus

  Cloning https://github.com/bp-kelley/descriptastorus to /tmp/pip-req-build-pxveyjfb
  Running command git clone --filter=blob:none --quiet https://github.com/bp-kelley/descriptastorus /tmp/pip-req-build-pxveyjfb
  Resolved https://github.com/bp-kelley/descriptastorus to commit da9760932ab9a78b116bc697795dd9e1798f087a
  Preparing metadata (setup.py) ... done
  Created wheel for descriptastorus: filename=descriptastorus-2.5.0.23-py3-none-any.whl size=1083540 sha256=e888d3837ff771ab22bd645bc7355490098758fc04eabedfa79c2bc64bdb8e7c
  Stored in directory: /tmp/pip-ephem-wheel-cache-ep3exjot/wheels/b0/91/ea/7e8b56f17611c9bebc7f08065799bd317abd64f066463e2e8a
Successfully built descriptastorus


In [6]:
pip install pandas-flavor

In [16]:
from DeepPurpose import utils, dataset
from DeepPurpose import DTI as models
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
from sklearn import metrics
import numpy as np
from sklearn.model_selection import train_test_split


In [17]:

#Load Data
################################################################


In [30]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [31]:
dt_08 = pd.read_csv('/content/drive/MyDrive/data/yamanishi_08/dt_all_08.txt',delimiter='\t',header=None)
dt_08.columns = ['head','relation','tail']

df_drug = pd.read_csv('/content/drive/MyDrive/data/yamanishi_08/791drug_struc.csv')
df_proseq = pd.read_csv('/content/drive/MyDrive/data/yamanishi_08/989proseq.csv')
df_proseq.columns = ['pro_id','pro_ids','seq']

pro_id = df_proseq['pro_id']


In [32]:

#define function
################################


In [33]:

def get_struc(data,df_drug,df_proseq):
    drug_struc = pd.merge(data,df_drug,how='left',left_on='head',right_on='drug_id')['smiles'].values
    pro_struc = pd.merge(data,df_proseq,how='left',left_on='tail',right_on='pro_id')['seq'].values
    return data['label'].values,drug_struc,pro_struc


def roc_auc(y,pred):
    fpr, tpr, _ = metrics.roc_curve(y, pred)
    roc_auc = metrics.auc(fpr, tpr)
    return roc_auc

def pr_auc(y, pred):
    precision, recall, _ = metrics.precision_recall_curve(y, pred)
    pr_auc = metrics.auc(recall, precision)
    return pr_auc


data_path = '/content/drive/MyDrive/data/yamanishi_08/data_folds/warm_start_1_10'
def load_data(i):
    train = pd.read_csv(data_path+'train_fold_'+str(i+1)+'.csv')
    test = pd.read_csv(data_path+'test_fold_'+str(i+1)+'.csv')
    return train,test


In [34]:

#need to be adjusted when changing methods
drug_encoding, target_encoding = 'MPNN', 'CNN'

def get_input(train_all,test_all):
    train_label = train_all['label']
    test_label = test_all['label']
    train_re, valid_re, y_train, y_valid = train_test_split(train_all[['head','relation','tail']],train_label,test_size=0.01,
                                                                random_state=0,
                                                                stratify=train_label)
    train_drug_feats,train_pro_feats = get_struc(train_re,df_drug,df_proseq)
    valid_drug_feats,valid_pro_feats = get_struc(valid_re,df_drug,df_proseq)
    test_drug_feats,test_pro_feats = get_struc(test_all,df_drug,df_proseq)
    train = utils.data_process(train_drug_feats, train_pro_feats, y_train,
                                drug_encoding, target_encoding,
                                split_method='no_split',
                                random_seed = 0)
    valid = utils.data_process(valid_drug_feats, valid_pro_feats, y_valid,
                            drug_encoding, target_encoding,
                            split_method='no_split',
                            random_seed = 0)
    test = utils.data_process(test_drug_feats, test_pro_feats, test_label,
                                drug_encoding, target_encoding,
                                split_method='no_split',
                                random_seed = 0)
    return train,valid,test



In [35]:

######################################## Training
#parameters for MPNN_CNN
config = utils.generate_config(drug_encoding = drug_encoding,
                         target_encoding = target_encoding,
                        cls_hidden_dims = [1024,1024,512],
                         train_epoch = 300,
                         LR = 0.001,
                         batch_size = 5000,
                         hidden_dim_drug = 128,
                         mpnn_hidden_size = 128,
                         mpnn_depth = 3,
                         cnn_target_filters = [32,64,64],
                         cnn_target_kernels = [4,8,8]
                        )


In [36]:

'''
#parameters for DeepDTI
config = utils.generate_config(drug_encoding, target_encoding,
                            cls_hidden_dims = [1024,1024,512],
                            train_epoch = 300,
                            LR = 0.001,
                            batch_size = 5000,
                            cnn_drug_filters = [32,64,96],
                            cnn_drug_kernels = [4,8,12],
                            cnn_target_filters = [32,64,96],
                            cnn_target_kernels = [4,8,12])
'''

for i in range(10):
    print(i)
    train,test = load_data(i)
    train_input,valid_input,test_input = get_input(train,test)
    model = models.model_initialize(**config)
    model.train(train_input,valid_input)


0


FileNotFoundError: ignored